In [1]:
# default_exp download

In [2]:
#exports
import pandas as pd
import typer
import textwrap
import requests
from bs4 import BeautifulSoup as bs

In [3]:
#exports
def get_every_noise_canvas(everynoise_url='https://everynoise.com/'):
    r = requests.get(everynoise_url)
    soup = bs(r.text, features='lxml')
    
    canvases = soup.find_all('div', attrs={'class': 'canvas'})
    assert len(canvases) == 1, ''
    canvas = canvases[0]

    return canvas

In [4]:
canvas = get_every_noise_canvas()

In [5]:
#exports
extract_style_elems = lambda genre_elem: {
    style_elem.split(': ')[0].strip(): style_elem.split(': ')[1].replace('px', '')
    for style_elem 
    in genre_elem['style'].split(';')
}

def extract_canvas_height_width(canvas):
    canvas_style_elems = extract_style_elems(canvas)

    canvas_height = int(canvas_style_elems['height'])
    canvas_width = int(canvas_style_elems['width'])

    return canvas_height, canvas_width

In [6]:
canvas_height, canvas_width = extract_canvas_height_width(canvas)

canvas_height, canvas_width

(19623, 1607)

In [7]:
genre_elem = canvas.find('div')

genre_elem

<div class="genre scanme" id="item1" onclick='playx("31GIhpOQpKCzlnuPn03pBQ", "piano blues", this);' preview_url="https://p.scdn.co/mp3-preview/fd161f29a12adc40d69671ad6ff05a20135dab40" scan="true" style="color: #66882d; top: 15753px; left: 911px; font-size: 101%" title="e.g. Champion Jack Dupree &quot;Junker's Blues&quot;">piano blues<a class="navlink" href="engenremap-pianoblues.html" onclick="event.stopPropagation();">»</a> </div>

In [8]:
#exports
genre_elem_to_name = lambda genre_elem: genre_elem.text.replace('» ', '')

In [9]:
genre_elem_to_name(genre_elem)

'piano blues'

In [10]:
genre_style_elems = extract_style_elems(genre_elem)

genre_style_elems

{'color': '#66882d', 'top': '15753', 'left': '911', 'font-size': '101%'}

In [11]:
#exports
def get_genre_xy(genre_style_elems, canvas_width, canvas_height):
    x = canvas_width - int(genre_style_elems['left'].replace('px', ''))
    y = canvas_height - int(genre_style_elems['top'].replace('px', ''))

    return x, y

In [12]:
x, y = get_genre_xy(genre_style_elems, canvas_width, canvas_height)

x, y

(696, 3870)

In [13]:
#exports
def hex_to_rgb(value):
    if value[0] == '#':
        value = value[1:]

    len_value = len(value)

    if len_value not in [3, 6]:
        raise ValueError('Incorect a value hex {}'.format(value))

    if len_value == 3:
        value = ''.join(i * 2 for i in value)
        
    return tuple(int(i, 16) for i in textwrap.wrap(value, 2))

In [14]:
rgb = hex_to_rgb(genre_style_elems['color'])

rgb

(102, 136, 45)

In [15]:
#exports
def extract_genre_attrs(genre_elem, canvas_width, canvas_height):
    genre_attrs = {}

    genre_style_elems = extract_style_elems(genre_elem)

    genre_attrs['genre'] = genre_elem_to_name(genre_elem)
    genre_attrs['x'], genre_attrs['y'] = get_genre_xy(genre_style_elems, canvas_width, canvas_height)
    genre_attrs['r'], genre_attrs['g'], genre_attrs['b'] = hex_to_rgb(genre_style_elems['color'])
    
    return genre_attrs

In [16]:
genre_attrs = extract_genre_attrs(genre_elem, canvas_width, canvas_height)

genre_attrs

{'genre': 'piano blues', 'x': 696, 'y': 3870, 'r': 102, 'g': 136, 'b': 45}

In [17]:
#exports
def get_df_genre_attrs(everynoise_url='https://everynoise.com/'):
    canvas = get_every_noise_canvas(everynoise_url=everynoise_url)
    canvas_height, canvas_width = extract_canvas_height_width(canvas)

    genre_elems = canvas.find_all('div')
    all_genre_attrs = []

    for genre_elem in genre_elems:
        genre_attrs = extract_genre_attrs(genre_elem, canvas_width, canvas_height)
        all_genre_attrs += [genre_attrs]
        
    df_genre_attrs = pd.DataFrame(all_genre_attrs)
    
    return df_genre_attrs

In [18]:
df_genre_attrs = get_df_genre_attrs()

df_genre_attrs.head()

,genre,x,y,r,g,b
0,piano blues,696,3870,102,136,45
1,australian classical piano,1105,613,28,163,194
2,histoire pour enfants,454,5661,86,134,8
3,dutch musical,1067,6080,118,140,20
4,blues,600,3971,105,134,32


In [19]:
#exports
app = typer.Typer()

In [25]:
#exports
@app.command()
def download_genre_attrs(fp='data/genre_attrs.csv'):
    df_genre_attrs = get_df_genre_attrs()
    df_genre_attrs.to_csv(fp, index=False)
    
    return

In [26]:
download_genre_attrs()

In [22]:
#exports
if __name__ == '__main__' and '__file__' in globals():
    app()

In [23]:
import os
from nbdev import export
from nbdev.imports import Path, Config

def create_mod_files(files, to_dict=False, bare=False):
    "Create mod files for default exports found in `files`"
    modules = []
    
    for f in sorted(files):
        fname = Path(f)
        nb = export.read_nb(fname)
        default = export.find_default_export(nb['cells'])
        
        if default is not None:
            default = os.path.sep.join(default.split('.'))
            modules.append(default)
            
            if not to_dict:
                nbs_path = Config().path("nbs_path")
                if nbs_path is None:
                    nbs_path = '.'
                    
                export.create_mod_file(Config().path("lib_path")/f'{default}.py', f'{nbs_path}/{fname}', bare=bare)
                
    return modules

export.create_mod_files = create_mod_files

In [27]:
#hide
export.notebook2script('library-gen.ipynb')

Converted library-gen.ipynb.
